In [1]:
import numpy as np
from scipy.io import loadmat
import nibabel as nb


from CLS_compute_eigs_all import Compute_Eigs
from CLS_dysco_distance import Dysco_distance
from CLS_dysco_reconf import Dysco_reconf_distance
from CLS_dysco_norm import Dysco_Norm

ModuleNotFoundError: No module named 'CLS_compute_eigs_all'

# Simple DySCO Tutorial 

### If you want to learn how to use dysco this is the right place! This script teaches you how to run the core functions to build your dysco analysis pipeline

## Step 1: Load the timeseries. 
### This should be a matrix, each row is a timepoint, each column is a signal/brain area/feature. So it's TxN. 

### This might be the most ~annoying~ crucial part, the data preprocessing/format. Here we will use .nii format and use nibabel to load. But there are many other ways to do this. 

In [3]:
file_path = 'Path to .nii file'

# Load NIfTI file
cifti = nb.load(file_path)
cifti_data = cifti.get_fdata(dtype=np.float32)
cifti_hdr = cifti.header
nifti_hdr = cifti.nifti_header

axes = [cifti_hdr.get_axis(i) for i in range(cifti.ndim)]

# Only using half the brain here 
left_brain = surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_LEFT')
# right_brain = surf_data_from_cifti(cifti_data, axes[1], 'CIFTI_STRUCTURE_CORTEX_RIGHT')

brain_load = left_brain

# Can filter here (based on tissue boundaries etc)
brain_load = brain_load.T
zero_columns = np.all(brain_load == 0, axis=0)
filtered_array = brain_load[:, ~zero_columns]
brain = filtered_array

FileNotFoundError: No such file or no access: 'Path to .nii file'

## Step 2: Run the recurrence matrix EVD

### After you have selected the type of matrix (see paper) and preprocessed the data, run the recurrence matrix EVD for the specified matrix. For example, here we are running it for a sliding window correlation matrix with a window of 21 (odd numbers for symmetry). Remember that the rank (=n of non-null eigenvalues) is lower than window size (see paper). In this case, we calculate the first 10 eigenvectors as an example.

In [4]:
half_window_size = 10
n_eigen = 10

instance = Compute_Eigs(brain, n_eigen, half_window_size)
eigenvectors, eigenvalues = inst.compute_eigs_cov()


NameError: name 'Compute_Eigs' is not defined

### Now you have eigenvectors and eigenvalues. eigenvalues is a 2D matrix, where each column corresponds to our 10 eigenvalues at each time point. eigenvectors is 3D, because it is a matrix of eigenvectors for each time point. Every column of the matrix is an eigenvector, and indeed every matrix has 10 columns.

## Step 3: Compute DySCo measures:

### Now that we have this EVD representation of our sliding-window correlation matrix, we can compute the DySCo measures.

#### These are: 
1. NORM 
2. DISTANCE 
3. Reconfiguration Speed 
4. Entropy


## Norm

### This is the time-varying norm, computed from eigenvalues (see paper), so at each time point you have the norm of the matrix. Let us compute the norm 2, but there are different norms available (see paper).

In [5]:
# norm1_inst = Dysco_Norm(1)
norm2_inst = Dysco_Norm(2)
# norminf_inst = Dysco_Norm(np.inf)

norm2 = norm2_inst.norm(eigval)


NameError: name 'Dysco_Norm' is not defined

In [6]:
# a1. From norm we can compute a derived measure, which is spectral
# metastability - see paper.
metastability = np.std(norm)

NameError: name 'norm' is not defined

## Distance

### We can compute the distance between dynamic matrices at 2 different time points. For example, let us use the distance 2 to compute the Functional Connectivity Dynamics (FCD) matrix.

In [7]:
T = eigenvectors.shape[0]
FCD = np.zeros((T, T))

eigs_inst = Dysco_distance(2)

for i in range(T):
    for j in range(i + 1, T):
        fcd_ij = eigs_inst.dysco_distance(eigenvectors[i, :, :], eigenvectors[j, :, :])
        fcd[i, j] = fcd_ij
        fcd[j, i] = fcd_ij

NameError: name 'eigenvectors' is not defined

## Reconfiguration speed

### is just the distance between the matrix at time t and the matrix at time t-lag, so if we already have the FCD matrix, the reconfiguration speed will be just derived from that: (here we suppose lag = 5)

In [8]:
lag = 5
speed = np.zeros(T - lag)
for i in range(T - lag):
    speed[i] = fcd[i, i + lag]
    

NameError: name 'T' is not defined

## Entropy

### For Von Neumann Entropy, you just need the eigenvalues (like for the norm).

In [ ]:
                      
entropy = eigenvalues / np.tile(np.sum(eigenvalues, axis=0), (n_eigen, 1))
entropy = -np.sum(np.log(entropy) * entropy, axis=0)
